Pontifícia Universidade Católica de São Paulo 

`Ciência de Dados e Inteligência Artificial`

🎓 Laboratório 7 - Classificador Naive Bayes (Parte 2)

---
> 👨‍🏫*Professor Rooney Coelho (rracoelho@pucsp.br)*
---


In [1]:
import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords
import string

import pandas as pd
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import naive_bayes

Carregue o dataset de SPAM para SMS

In [2]:
df = pd.read_csv('sms_spam.csv', encoding='latin1')
df = df[['v1','v2']]
df.columns = ['categoria', 'mensagem']
df

,categoria,mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


### Pré-processamento

Traduza a coluna `categoria` para `ham: 0` e `spam: 1` use o método do Pandas adequado para isto.

In [3]:
codes, uniques = pd.factorize(df['categoria'])
df['categoria'] = codes

In [29]:
df['categoria']

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: categoria, Length: 5572, dtype: int64

Use a função abaixo para criar uma nova coluna no dataset para o texto pré-processado.

1. Remoção de pontuação
2. Passar o texto para caixa baixa
3. Remover palavras vazias
4. Remover dígitos

In [4]:
def preprocessText(msg):
    removepunc = [c for c in msg if c not in string.punctuation]
    removepunc = "".join(removepunc)
    lower = removepunc.lower()
    tokenize = nltk.word_tokenize(lower)
    remove_stop = [word for word in tokenize if word not in stopwords.words('english')]
    remove_digit = [word for word in remove_stop if not word.isdigit()]
    joinagain = " ".join(remove_digit)
    return joinagain

Crie a coluna `mensagem_pre` e use o método `apply()` do Pandas para varrear o Dataframe com a função definida.

In [5]:
df['mensagem_pre'] = df['mensagem'].apply(preprocessText)

### Extração de Features

Importe o `CountVectorizer` do Scikit-Learn

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

Inicialize o Bag of Words para as mensagens pré-processadas.

Dica:
```python
vectorizer = CountVectorizer()
vectorizer.fit(df[coluna].values)
```

In [8]:
vectorizer = CountVectorizer()
vectorizer.fit(df['mensagem_pre'].values)

CountVectorizer()

Crie as features a serem usadas na classificação, armazenadas em `X`.

Dica:
```python
X = vectorizer.transform(df[coluna].values)
```

In [9]:
X = vectorizer.transform(df['mensagem_pre'].values)

Chame o target, que é a categoria, de `y`. 

In [10]:
y = df['categoria']

### Segmentação dos dados

Realize a segmentação dos dados usando o `train_test_split`. Separe 20% dos dados para teste.

In [11]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Treinamento do classificador

Treine o classificador Multinomial Naive Bayes com a partição feita.

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
mnnb_model = MultinomialNB()

In [20]:
mnnb_model.fit(X_train, y_train)

MultinomialNB()

Avalie a precisão de seu classificador usando o `score`.

In [21]:
mnnb_model.score(X_test, y_test)

0.968609865470852

### Avalie agora o seu modelo nas seguintes mensagens:

In [25]:
msg1 = 'Congratulations! You`ve won a $1,000 Walmart gift card. Go to http://bit.ly/123456 to claim now.'
msg2 = 'Hello son, when are you coming back home? Mommy'

* Não se esqueça de fazer o pré-processamento para inserir no modelo.

dica:
```python
vectorizer.transform([texto_preprocessado])
```

In [26]:
msgs = pd.Series([msg1, msg2])

In [27]:
# Aplicar preprocessText
msgs_pre = msgs.apply(preprocessText)
# Vetorizar
X = vectorizer.transform(msgs_pre)

In [28]:
mnnb_model.predict(X)

array([1, 0], dtype=int64)

<i>Baseado no modelo que nós treinamos, <b>a primeira</b> mensagem seria classificada como <b>spam</b> e <b>a segunda não</b></i>